# SageMaker Calculator Model Deployment

This notebook deploys the calculator model to SageMaker for real-time inference.

In [ ]:
import boto3
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role
import tarfile
import os
import json

In [ ]:
# Endpoint cleanup function
def delete_existing_endpoint(endpoint_name):
    import time
    try:
        sagemaker_client = boto3.client('sagemaker')
        sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
        print(f'Deleting existing endpoint: {endpoint_name}')
        
        while True:
            try:
                sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
                print('Waiting for endpoint deletion...')
                time.sleep(10)
            except:
                print('Endpoint deleted successfully')
                break
    except:
        print(f'No existing endpoint found: {endpoint_name}')

In [ ]:
# Initialize SageMaker session and get role
sagemaker_session = sagemaker.Session()
role = get_execution_role()
print(f"SageMaker role: {role}")
print(f"Default bucket: {sagemaker_session.default_bucket()}")

In [ ]:
# Delete existing endpoint before deployment
delete_existing_endpoint('math-calculator-endpoint')

In [ ]:
# Create model package
os.makedirs('code', exist_ok=True)

# Copy model files
import shutil
shutil.copy('../src/calculator_model.py', 'code/')
shutil.copy('../src/inference.py', 'code/')

# Create model.tar.gz
with tarfile.open('model.tar.gz', 'w:gz') as tar:
    tar.add('code', arcname='code')

print("Model package created: model.tar.gz")

In [ ]:
# Upload model to S3
bucket = sagemaker_session.default_bucket()
model_artifacts = sagemaker_session.upload_data(
    path='model.tar.gz',
    bucket=bucket,
    key_prefix='calculator-model'
)

print(f"Model uploaded to: {model_artifacts}")

In [ ]:
# Create PyTorch model
pytorch_model = PyTorchModel(
    model_data=model_artifacts,
    role=role,
    entry_point='inference.py',
    source_dir=None,
    framework_version='1.12',
    py_version='py38'
)

print("PyTorch model created")

In [ ]:
# Deploy to endpoint
predictor = pytorch_model.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium',
    endpoint_name='math-calculator-endpoint'
)

print(f"Model deployed to endpoint: {predictor.endpoint_name}")

In [ ]:
# Configure predictor for JSON
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()

# Test the endpoint
test_data = {
    "operation": "add",
    "a": 10,
    "b": 5
}

try:
    result = predictor.predict(test_data)
    print(f"Test result: {result}")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# Test multiple operations
test_cases = [
    {"operation": "add", "a": 10, "b": 5},
    {"operation": "multiply", "a": 4, "b": 3},
    {"operation": "sqrt", "a": 16},
    {"operation": "sin", "a": 90}
]

for test in test_cases:
    result = predictor.predict(test)
    print(f"Input: {test} -> Result: {result}")

In [ ]:
# Logging function
def get_endpoint_logs(endpoint_name):
    from datetime import datetime, timedelta
    
    logs_client = boto3.client('logs')
    log_group = f'/aws/sagemaker/Endpoints/{endpoint_name}'
    
    try:
        end_time = datetime.now()
        start_time = end_time - timedelta(minutes=10)
        
        response = logs_client.filter_log_events(
            logGroupName=log_group,
            startTime=int(start_time.timestamp() * 1000),
            endTime=int(end_time.timestamp() * 1000)
        )
        
        print(f'=== Endpoint Logs for {endpoint_name} ===')
        for event in response['events']:
            timestamp = datetime.fromtimestamp(event['timestamp'] / 1000)
            print(f'[{timestamp}] {event["message"]}')
    except Exception as e:
        print(f'Could not fetch logs: {e}')

In [ ]:
# View endpoint logs
get_endpoint_logs(predictor.endpoint_name)

In [ ]:
# Save endpoint name for later use
endpoint_name = predictor.endpoint_name
print(f"Endpoint name: {endpoint_name}")

# Save to file
with open('endpoint_config.json', 'w') as f:
    json.dump({'endpoint_name': endpoint_name}, f)
    
print("Endpoint configuration saved to endpoint_config.json")

In [ ]:
# Cleanup endpoint (run when done)
# predictor.delete_endpoint()
# print('Endpoint deleted')